In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [93]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score




from sklearn.model_selection import train_test_split
import os
import glob
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.layers import Input
from keras import Sequential
from keras.layers import Dense, Input
from keras.layers import Permute, Reshape, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Bidirectional, GRU, TimeDistributed, Dense, Conv1D, LSTM,  Flatten
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
import keras_metrics
import keras.backend as K
from datetime import datetime
from sklearn.utils import class_weight
from tensorflow.keras.models import load_model

In [3]:

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = np.expand_dims(x_train, axis=-1) / 255.0
x_test = np.expand_dims(x_test, axis=-1) / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


11490434/11490434 [==============================] - 1s 0us/step


In [4]:

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])




In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)


Epoch 1/5
422/422 [==============================] - 24s 55ms/step - loss: 0.2713 - precision: 0.9602 - recall: 0.8822 - val_loss: 0.0809 - val_precision: 0.9807 - val_recall: 0.9752
Epoch 2/5
422/422 [==============================] - 22s 52ms/step - loss: 0.0707 - precision: 0.9814 - recall: 0.9755 - val_loss: 0.0498 - val_precision: 0.9885 - val_recall: 0.9845
Epoch 3/5
422/422 [==============================] - 21s 50ms/step - loss: 0.0501 - precision: 0.9863 - recall: 0.9829 - val_loss: 0.0402 - val_precision: 0.9910 - val_recall: 0.9892
Epoch 4/5
422/422 [==============================] - 21s 50ms/step - loss: 0.0406 - precision: 0.9890 - recall: 0.9864 - val_loss: 0.0420 - val_precision: 0.9916 - val_recall: 0.9885
Epoch 5/5
422/422 [==============================] - 21s 49ms/step - loss: 0.0323 - precision: 0.9910 - recall: 0.9888 - val_loss: 0.0478 - val_precision: 0.9865 - val_recall: 0.9850


In [6]:

# Evaluate the model
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')

print("Precision:", precision)
print("Recall:", recall)

313/313 [==============================] - 2s 6ms/step
Precision: 0.987404083374771
Recall: 0.9872937604680146


In [7]:
np.dot(y_train, np.array([i for i in range(10)]))

array([5., 0., 4., ..., 5., 6., 8.])

In [22]:
x_train_new, y_train_new = [], []
x_train_base_new, y_train_base_new = [], []
labels = np.dot(y_train, np.array([i for i in range(10)]))

unknown_labels = [7, 8, 9]

for x, y, l in zip(x_train, y_train, labels):
    if l in unknown_labels:
        x_train_new.append(x)
        y_train_new.append(y[7:])
    else:
        x_train_base_new.append(x)
        y_train_base_new.append(y[:7])
x_train_new, y_train_new = np.array(x_train_new), np.array(y_train_new)
x_train_base_new, y_train_base_new = np.array(x_train_base_new), np.array(y_train_base_new)

In [23]:
x_train_new.shape, x_train_base_new.shape, y_train_new.shape, y_train_base_new.shape

((18065, 28, 28, 1), (41935, 28, 28, 1), (18065, 3), (41935, 7))

In [24]:
x_test_new, y_test_new = [], []
x_test_base_new, y_test_base_new = [], []
labels = np.dot(y_test, np.array([i for i in range(10)]))


for x, y, l in zip(x_test, y_test, labels):
    if l in unknown_labels:
        x_test_new.append(x)
        y_test_new.append(y[7:])
    else:
        x_test_base_new.append(x)
        y_test_base_new.append(y[:7])
x_test_new, y_test_new = np.array(x_test_new), np.array(y_test_new)
x_test_base_new, y_test_base_new = np.array(x_test_base_new), np.array(y_test_base_new)

In [25]:
x_test_new.shape, x_test_base_new.shape, y_test_new.shape, y_test_base_new.shape

((3011, 28, 28, 1), (6989, 28, 28, 1), (3011, 3), (6989, 7))

In [104]:

# Build the CNN model
Model = Sequential()
Model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
Model.add(MaxPooling2D((2, 2)))
Model.add(Conv2D(64, (3, 3), activation='relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Flatten())
# Model.add(Dense(64, activation='relu'))
Model.add(Dense(10-len(unknown_labels), activation='softmax'))

# Compile the model
Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [105]:
# Train the model
Model.fit(x_train_base_new, y_train_base_new, batch_size=128, epochs=5, validation_split=0.1)


Epoch 1/5
295/295 [==============================] - 15s 49ms/step - loss: 0.2231 - precision_3: 0.9730 - recall_3: 0.8878 - val_loss: 0.0525 - val_precision_3: 0.9899 - val_recall_3: 0.9852
Epoch 2/5
295/295 [==============================] - 15s 50ms/step - loss: 0.0443 - precision_3: 0.9878 - recall_3: 0.9847 - val_loss: 0.0430 - val_precision_3: 0.9912 - val_recall_3: 0.9893
Epoch 3/5
295/295 [==============================] - 14s 48ms/step - loss: 0.0322 - precision_3: 0.9908 - recall_3: 0.9888 - val_loss: 0.0351 - val_precision_3: 0.9919 - val_recall_3: 0.9905
Epoch 4/5
295/295 [==============================] - 15s 50ms/step - loss: 0.0247 - precision_3: 0.9930 - recall_3: 0.9918 - val_loss: 0.0353 - val_precision_3: 0.9924 - val_recall_3: 0.9917
Epoch 5/5
295/295 [==============================] - 14s 48ms/step - loss: 0.0212 - precision_3: 0.9938 - recall_3: 0.9924 - val_loss: 0.0258 - val_precision_3: 0.9947 - val_recall_3: 0.9936


In [106]:
Model.evaluate(x_test_base_new, y_test_base_new, batch_size = 128)

55/55 [==============================] - 1s 14ms/step - loss: 0.0188 - precision_3: 0.9947 - recall_3: 0.9937


[0.018811633810400963, 0.9947006702423096, 0.9937043786048889]

In [107]:
x_test_base_new.shape, y_test_base_new.shape

((6989, 28, 28, 1), (6989, 7))

In [108]:
x_train_new.shape

(18065, 28, 28, 1)

In [109]:
Model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 7)                

In [110]:
newModel = Sequential(Model.layers[:-2])

In [111]:
newModel.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [113]:
newModel.predict(x_test_base_new).shape

219/219 [==============================] - 1s 5ms/step


(6989, 5, 5, 64)

In [116]:
# Build the CNN model
metaModel = Sequential()
metaModel.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(5, 5, 64)))
metaModel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
metaModel.add(Flatten())
metaModel.add(Dropout(0.5))  # Adding a dropout layer with a rate of 0.5
metaModel.add(Dense(3, activation='softmax'))

# Compile the model
metaModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [117]:
metaModel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 3, 3, 32)          18464     
                                                                 
 conv2d_11 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 37,155
Trainable params: 37,155
Non-trainable params: 0
_________________________________________________________________


In [148]:
# 3 way 5 shot setup
def task():
    num_tasks = 20
    tasks = [[] for _ in range(num_tasks)]
    labels = np.dot(y_train_new, np.array([i for i in range(7, 10)]))
    x, y = [[] for _ in unknown_labels], [[] for _ in unknown_labels]
    for a, b in zip(x_test_new, y_test_new):
        x[np.argmax(b)].append(a)
        y[np.argmax(b)].append(b)
        
    X = [np.array(i) for i in x]
    Y = [np.array(i) for i in y]
    del(x, y)
    
    for task in range(num_tasks):
        tasks[task].append(np.concatenate([X[i][task*5:(task+1)*5] for i in range(len(X))]))
        tasks[task].append(np.concatenate([Y[i][task*5:(task+1)*5] for i in range(len(Y))]))
        tasks[task].append(np.concatenate([X[i][num_tasks*5+task*2:num_tasks*5+(task+1)*2] for i in range(len(X))]))
        tasks[task].append(np.concatenate([Y[i][num_tasks*5+task*2:num_tasks*5+(task+1)*2]for i in range(len(Y))]))
    return tasks
    

In [149]:
task_list = task()

In [157]:
metaModel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 3, 3, 32)          18464     
                                                                 
 conv2d_11 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 37,155
Trainable params: 37,155
Non-trainable params: 0
_________________________________________________________________


In [150]:
for task in task_list:
    print(task[0].shape, task[1].shape, task[2].shape, task[3].shape)

(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)


In [ ]:
metaModel = Sequential()
metaModel.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(5, 5, 64)))
metaModel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
metaModel.add(Flatten())
metaModel.add(Dropout(0.5))  # Adding a dropout layer with a rate of 0.5
metaModel.add(Dense(3, activation='softmax'))

# Compile the model
metaModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [138]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
precision_metric = keras.metrics.Precision()
recall_metric = keras.metrics.Recall()
optim = tf.keras.optimizers.legacy.Adam(
    learning_rate=0.001
)

for epoch in range(12):
    print(f"start of epoch : {epoch}")
    outer_loss = None
    
    with tf.GradientTape(persistent = True) as tape:
        for task in task_list:
            y_pred = metaModel(newModel(task[0], training = False), training = True)
            loss = loss_fn(task[1], y_pred)

            y_pred_new = metaModel(newModel(task[2], training = True), training = True)
            loss_new = loss_fn(task[3], y_pred_new)
            if outer_loss == None:
                outer_loss = loss_new
            else:
                outer_loss += loss_new
            gradients = tape.gradient(loss, metaModel.trainable_weights)
            optim.apply_gradients(zip(gradients, metaModel.trainable_weights))
        newGradients = tape.gradient(outer_loss, newModel.trainable_weights)
        optim.apply_gradients(zip(newGradients, newModel.trainable_weights))
        
        
        
        
        
    

start of epoch : 0
start of epoch : 1
start of epoch : 2
start of epoch : 3
start of epoch : 4
start of epoch : 5
start of epoch : 6
start of epoch : 7
start of epoch : 8
start of epoch : 9
start of epoch : 10
start of epoch : 11


In [119]:
metaModel.predict(newModel.predict(x_test_base_new)).shape

219/219 [==============================] - 0s 2ms/step


(6989, 3)

TensorShape([15, 3])

In [126]:
 metaModel(newModel(task_list[0][0], training = False), training = True)

TensorShape([15, 3])

In [143]:
print(metaModel(newModel(np.array([task_list[0][0][10]]), training = False), training = False))

tf.Tensor([[4.8072208e-04 1.5938188e-03 9.9792540e-01]], shape=(1, 3), dtype=float32)


In [144]:
print(task_list[0][1][10])

[0. 0. 1.]
